In [ ]:
!nvidia-smi

Wed Feb 21 10:12:56 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   50C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model

trdata = ImageDataGenerator()
traindata = trdata.flow_from_directory('/content/drive/MyDrive/KIMIA_PATH/Training100', target_size=(224,224), batch_size=32, shuffle=False)

Found 960 images belonging to 20 classes.


In [ ]:
import tensorflow as tf

model = tf.keras.applications.VGG19(
    include_top = False,
    weights="imagenet",
    input_shape=(224,224,3),
    pooling = max
)

80134624/80134624 [==============================] - 0s 0us/step


In [ ]:
model.summary()

Model: "vgg19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [ ]:
#start a keras sequential model
from keras.layers import Flatten, Dense, Dropout
from keras.models import Sequential
new_model = Sequential()

# add the convolutional part of the VGG19 model from above
new_model.add(model)

# flatten the output of the VGG16 model because it is from a convolutional layer
new_model.add(Flatten())

# add a dense (aka. fully-connected) layer
# this is for combining features that the VGG19 model has
# recognized in the image.
new_model.add(Dense(256, activation='relu'))

# add a dropout-layer which may prevent overfitting and improve generalization
# ability to unseen data e.g. the test-set.
new_model.add(Dropout(0.5))

# add the final layer for the actual classifcation.
# 20 because we have 20 classes
new_model.add(Dense(20,activation='softmax'))

In [ ]:
for layer in new_model.layers:
  print(layer.name, layer.trainable)

vgg19 True
flatten True
dense True
dropout True
dense_1 True


In [ ]:
new_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg19 (Functional)          (None, 7, 7, 512)         20024384  
                                                                 
 flatten (Flatten)           (None, 25088)             0         
                                                                 
 dense (Dense)               (None, 256)               6422784   
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 20)                5140      
                                                                 
Total params: 26452308 (100.91 MB)
Trainable params: 26452308 (100.91 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
from keras.metrics import Precision, Recall
from keras import optimizers, losses

myAdam = optimizers.Adam(learning_rate = 0.00001)
new_model.compile(loss='categorical_crossentropy', optimizer = myAdam , metrics=['accuracy', Precision(), Recall()])

#Train the Model

In [ ]:
hist = new_model.fit(traindata,
                     epochs = 30,
                     steps_per_epoch=len(traindata),
                     )

Epoch 1/30
30/30 [==============================] - 422s 14s/step - loss: 9.1611 - accuracy: 0.0542 - precision: 0.0656 - recall: 0.0375
Epoch 2/30
30/30 [==============================] - 14s 452ms/step - loss: 3.2917 - accuracy: 0.0646 - precision: 0.1364 - recall: 0.0031
Epoch 3/30
30/30 [==============================] - 14s 458ms/step - loss: 2.8735 - accuracy: 0.1292 - precision: 0.1250 - recall: 0.0010
Epoch 4/30
30/30 [==============================] - 14s 464ms/step - loss: 2.6198 - accuracy: 0.1969 - precision: 0.7895 - recall: 0.0156
Epoch 5/30
30/30 [==============================] - 14s 466ms/step - loss: 2.3782 - accuracy: 0.2771 - precision: 0.7500 - recall: 0.0531
Epoch 6/30
30/30 [==============================] - 14s 467ms/step - loss: 2.1236 - accuracy: 0.3521 - precision: 0.8435 - recall: 0.1010
Epoch 7/30
30/30 [==============================] - 14s 469ms/step - loss: 1.8316 - accuracy: 0.4500 - precision: 0.8070 - recall: 0.1917
Epoch 8/30
30/30 [=================

In [ ]:
import numpy as np
import os
import cv2
import pandas as pd

dense_layer = 'dense'
feature_model = Model(inputs = new_model.input, outputs = new_model.get_layer(dense_layer).output)

image_size = 224
features_list = []
filenames_list = []

labels = ['Class_A','Class_B','Class_C','Class_D','Class_E','Class_F','Class_G','Class_H','Class_I','Class_J','Class_K','Class_L','Class_M','Class_N','Class_O','Class_P','Class_Q','Class_R','Class_S','Class_T']
for i in labels:
    folderPath = os.path.join('/content/drive/MyDrive/KIMIA_PATH/Training100',i)
    for j in sorted(os.listdir(folderPath)):
        img = cv2.imread(os.path.join(folderPath,j))
        img = cv2.resize(img,(image_size,image_size))

        features = feature_model.predict(np.expand_dims(img, axis=0))
        filenames_list.append(i)
        features_list.append(features.flatten())

features_df = pd.DataFrame(features_list)
features_df['Filename'] = filenames_list

# features_df.to_excel('vgg19_features.xlsx', index = False)
